# HSQC fitting

## Module import 

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from scipy.optimize import curve_fit
plt.ion()
from sklearn.linear_model import LinearRegression
import math
from matplotlib import rc, rcParams
from mpl_toolkits.mplot3d import Axes3D 
from scipy import stats

## Parameters input:

In [19]:
m_lig = 51.7 #mass of material (extracted lignin/ cell wall) in mg
m_PS = 7.1 #mass of polystyrene standard in mg
MW_lig = #molecular weight of lignin repetiting unit 
MW_PS = 102  #molecular weight of polystyrene standard repetiting unit 


## Functions generation:

In [20]:
# Generate a list of local maxima based on neighbouring intensity. The output is sorted in descending intensity order.
def detect_local_maxima(data, x_names, y_names, threshold = 0):
    local_max = []
    for j in range(1, data.shape[1] - 1):
        for i in range(1, data.shape[0] - 1):
            middle = data[i][j]
            top = data[i][j - 1]
            bottom = data[i][j + 1]
            left = data[i - 1][j]
            right = data[i + 1][j]
            if (middle > top and
                middle > bottom and
                middle > left and
                middle > right and
                middle > threshold):
                # (intensity, x_index, y_index)
                local_max += [(middle, i, j, x_names[j], y_names[i] )]
    local_max = sorted(local_max)
    local_max.reverse()
    return local_max

# 2D Gaussian function with independent components.
def gauss2d(Z, A, mu_x, mu_y, sigma_x, sigma_y):
    x = np.array(Z[0])
    y = np.array(Z[1])
    x_exp = np.exp(-0.5 * ((x - mu_x)/sigma_x) ** 2)
    y_exp = np.exp(-0.5 * ((y - mu_y)/sigma_y) ** 2)
    ret = (A * x_exp * y_exp)
    return ret.ravel()

def calculate_r2(X, Y, Z, popt):
    Z_fit = gauss2d((X, Y), popt[0], popt[1], popt[2], popt[3], popt[4])
    Z = Z.ravel()
    residuals = Z - Z_fit
    print (residuals)
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((Z - np.mean(Z)) ** 2)
    print(ss_tot, ss_res)
    R2 = 1 -(ss_res / ss_tot)
    return R2

def calculate_r2_2(X, Y, Z, params):
    Z_fit = gauss2d_2((X, Y), *popt)
    Z = Z.ravel()
    residuals = Z - Z_fit
    print('residuals', residuals)
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((Z - np.mean(Z)) ** 2)
    print(ss_tot, ss_res)
    R2 = 1 -(ss_res / ss_tot)
    return R2

def gauss_1d_x (x, A, x_0, sigma_x):
  C =  A*(1/(sigma_x*(np.sqrt(2*np.pi))))*(np.exp((-1.0/2.0)*(((x-x_0)/sigma_x)**2)))
  return C #return the gaussian fitting 


# 2D Gaussian function with overlapping.
def gauss2d_2(Z, A, mu_x, mu_y, sigma_x, sigma_y, B, mu_x_2, mu_y_2, sigma_x_2, sigma_y_2):
    x = np.array(Z[0])
    y = np.array(Z[1])
    x_exp = np.exp(-0.5 * ((x - mu_x)/sigma_x) ** 2) 
    x_exp_2 = np.exp(-0.5 * ((x - mu_x_2)/sigma_x_2) ** 2)
    y_exp = np.exp(-0.5 * ((y - mu_y)/sigma_y) ** 2)
    y_exp_2 = np.exp(-0.5 * ((y - mu_y_2)/sigma_y_2) ** 2)
    ret = (A * x_exp * y_exp + B * x_exp_2 * y_exp_2)
    return ret.ravel()



## Data loading:   

In [31]:
#Loading the three HSQC text file previously extracted witg rbnmr or NMRpipe
data_1 = pd.read_csv('/Users/xxx/Documents/nmr/Birch_1/1/pdata/1/Data.txt', sep='\t', header=None) #HSQC_1
data_2 = pd.read_csv('/Users/xxx/Documents/nmr/Birch_1/2/pdata/1/Data.txt', sep='\t', header=None) #HSQC_2
data_3 = pd.read_csv('/Users/xxx/Documents/nmr/Birch_1/2/pdata/1/Data.txt', sep='\t', header=None) #HSQC_3
x = pd.read_csv('/Users/xxx/Documents/nmr/Birch_1/2/pdata/1/X.txt', sep = "\t",  header=None) #x axis - 1H (ppm)
y = pd.read_csv('/Users/xxx/Documents/nmr/Birch_1/2/pdata/1/Y.txt', sep = "\t", header=None) #y axis - 1H (ppm)

Data_1 = data_1.values 
Data_2 = data_2.values
Data_3 = data_3.values
data = [] #creating one big data set with three spectra
data.append(Data_1)
data.append(Data_2)
data.append(Data_3)

X = x.values
Y = y.values
x_names = X.flatten()
y_names = Y.flatten()

### Preliminary results:

In [ ]:
# Plot a 2D image of the matrix
hfont = {'fontname':'Arial'}
ax = plt.pcolormesh(x_names, y_names, data_1, vmin=-10., vmax=10., shading='auto', cmap='hsv')
ax = plt.axes()
ax.invert_xaxis()
ax.invert_yaxis()
ax.yaxis.tick_right()
ax.text('1H (ppm)', '13C (ppm)', 'Intensity')
plt.xlabel('1H (ppm)', **hfont)
plt.ylabel('13C (ppm)', **hfont)
plt.show()

## Data processing

### Extract peaks of interest out of the NMR spectra

In [ ]:
results = []
HSQC = np.array([1,2,3])

for h in range (0, (len(data)), 1): 
  data_df = pd.DataFrame(data[h], columns=x_names, index=y_names)  #Open HSQC spectra successively 
  print("Detecting local maxima")
  local_max = detect_local_maxima(data[h], x_names, y_names, 1e3)
  print("> found: ", len(local_max))
  #print (local_max)
  
  #Create as many lists of indexes as peaks you want to analyse 
  ind = [] 
  ind_PS = [] #standard peak
  ind_Alpha = [] #B-O-4 alpha peak
  ind_Beta = [] #B-O-4 beta peak
    
  for k in range (0, len(local_max)): #look for our peaks of interest: alpha, beta, gamma, acetal, IS 
  #specify in which region (x_up, x_down, y_up, y_down) you can find your peak of interest.
  #do it for all the peaks defined above
    if (local_max[k][4] >= y_down and local_max[k][4] <= y_up and local_max[k][3] >= x_down and local_max[k][3] <= x_up):
      ind_Alpha.append(k)
    elif (local_max[k][4] >= 39 and local_max[k][4] <= 43 and local_max[k][3] >= 1.7 and local_max[k][3] <= 2.3): 
      ind_PS.append(k)
  print('PS', ind_PS, 'Alpha', ind_Alpha, 'Beta', ind_Beta)


  #Select the highest local maxima as the peak to be fitted for each peak of interest
  #Do it if the region you have specified is big and several peaks have been found
  int_Alpha = [local_max[q][0] for q in ind_Alpha]
  max_a = np.max(int_Alpha)
  index_a = np.where(int_Alpha == max_a)
  index_a = index_a[0]
  print(index_a)
  ind.append(ind_Alpha[index_a[0]])

  int_Beta = [local_max[q][0] for q in ind_Beta]
  max_b = np.max(int_Beta)
  index_b = np.where(int_Beta == max_b)
  index_b = index_b[0]
  ind.append(ind_Beta[index_b[0]])

  #Always put the standard as last item of the list
  int_PS = [local_max[q][0] for q in ind_PS]
  max_PS = np.max(int_PS)
  index_PS = np.where(int_Apo == max_PS)
  index_PS = index_PS[0]
  ind.append(ind_PS[index_PS[0]])

  print(ind)

  candidate_peaks=[]
  #Adapt the following list depending on the number of peaks fitted
  candidate_peaks = list([local_max[ind[0]], local_max[ind[1]], local_max[ind[2]]])
  #Generate the final list of peaks we will work on
  print(candidate_peaks)

### Analysis and gaussian fitting of local maxima: extracting peaks data

In [ ]:
# Start the analysis of the selected N local maxima.
# Define a window we will work on to fit the n peaks - to be adapted and reduced in case of overlaping peaks
  roi_n_samples_x_min = [[6, 6, 6], [6, 6, 6], [6, 6, 6]]  
  roi_n_samples_x_max = [[6, 6, 6], [6, 6, 6], [6, 6, 6]]
  roi_n_samples_y = [6, 6, 6]

    
  #Confidence interval
  ci = 0.95
  # Convert to percentile point of the normal distribution.
  # See: https://en.wikipedia.org/wiki/Standard_score
  pp = (1. + ci) / 2.
  # Convert to number of standard deviations.
  nstd = stats.norm.ppf(pp)
  print (nstd)
  
  plt.close('all')
  plt.ioff()
  fitted_params = []
  param_names = [
    "local_max_x",
    "local_max_y",
    "local_max_intensity",
    "fitted_height",
    "fitted_x",
    "fitted_y",
    "fitted_sigma_x",
    "fitted_sigma_y",
    "fitted_volume",
    "fit_R2"]
  
  for (k, candidate) in enumerate(candidate_peaks):
    intensity, i, j, x_h, y_c = candidate
    x0 = data_df.columns[j]
    y0 = data_df.index[i]
    print(k)

    # Calculate the region of interest (ROI) for this candidate peak
    min_x = j - roi_n_samples_x_min[h][k]
    max_x = j + roi_n_samples_x_max[h][k]
    min_y = i - roi_n_samples_y[k]
    max_y = i + roi_n_samples_y[k]
    peak_data = data_df.iloc[min_y:max_y, min_x:max_x] #ROI
    print(peak_data)
    peak_data_red = data_df.iloc[min_y:max_y, min_x:max_x]

    # Prepare the parameter matrix for curve fit.
    x_values = peak_data.columns
    y_values = peak_data.index
    print(x_values)
    X, Y = np.meshgrid(x_values, y_values)

    # Initial fit parameters.
    p0 = (intensity, x0, y0, 0.01, 0.05)
    print(p0)

    # The bounds can be used to constrain each parameter fitting to a range.
    tol_x = 0.5
    tol_y = 0.5
    bounds = (
        (1e3, x0 - tol_x, y0 - tol_y, 0.001, 0.01),
        (1e12, x0 + tol_x, y0 + tol_y, 1, 50)
        )
        
    # If fitting fails for this candidate peak, we move on to the next one.
    try: # Do the thing!
      popt, pcov = curve_fit(gauss2d, (X, Y), peak_data_red.values.ravel(), p0=p0, bounds=bounds)
      print(popt)
      # Standard deviation errors on the parameters.
      perr = np.sqrt(np.diag(pcov))
      # Add nstd standard deviations to parameters to obtain the upper confidence interval.
      popt_up = popt + nstd * perr
      popt_dw = popt - nstd * perr
    except:
      print('Failed')
      continue
    
    # Prepare the fitted data for a contour plotting. We use 100 points for higher resolution of circles, which looks nicer.
    x_fit = np.linspace(np.min(x_values), np.max(x_values), 100)
    y_fit = np.linspace(np.min(y_values), np.max(y_values), 100)
    X_fit, Y_fit = np.meshgrid(x_fit, y_fit)
    peak_data_fit = gauss2d((X_fit, Y_fit), *popt).reshape(100, 100)

     Calculate volume.
    # If using the overlaping function: 
    # fit_height, fit_x0, fit_y0, fit_sigma_x, fit_sigma_y, fit_height_2, fit_x0_2, fit_y0_2, fit_sigma_x_2, fit_sigma_y_2) = popt
    (fit_height, fit_x0, fit_y0, fit_sigma_x, fit_sigma_y) = (popt[0], popt[1], popt[2], popt[3], popt[4])
    fit_volume = fit_height * fit_sigma_x * fit_sigma_y * 2.0 * np.pi 
        
    # Calculate R2, but only within N * sigma in each direction, since multiple peaks may appear in the ROI.
    n_sig_x = 4 #adapt the value based on your data
    n_sig_y = 300
    x_index = ((peak_data.columns > x0 - fit_sigma_x * n_sig_x) &
                (peak_data.columns < x0 + fit_sigma_x * n_sig_x))
    y_index = ((peak_data.index > y0 - fit_sigma_y * n_sig_y) &
               (peak_data.index < y0 + fit_sigma_y * n_sig_y))
        

    x_r2 = peak_data.columns[x_index]
    y_r2 = peak_data.index[y_index]
    X_r2, Y_r2 = np.meshgrid(x_r2, y_r2)
    Z_r2 = peak_data_red.iloc[y_index, x_index]

   # peak_data
   fit_r2 = calculate_r2(X_r2, Y_r2, Z_r2.values, popt) #use calculate_r2_2 if use of gauss2d_2
   fitted_params = [(h+1), x0, y0, intensity, popt[0], popt[1], popt[2], popt[3], popt[4], fit_volume, np.log(fit_volume), fit_r2, popt_up[0], popt_up[1], popt_up[2], popt_up[3], popt_up[4], popt_down[0], popt_down[1], popt_down[2], popt_down[3], popt_down[4]] #, fit_r2)]
   print(fitted_params)
   results.append(fitted_params)

## Data plotting: 2D / 3D with corresponding fit. 

   # Plot the candidate peaks in 2D and the corresponding fit.
    fig = plt.figure()
    ax = plt.axes()
    ax.pcolormesh(peak_data.columns, peak_data.index, peak_data, shading='auto')
    ax.contour(X_fit, Y_fit, peak_data_fit, 5, colors='w', alpha=0.8)
    ax.invert_xaxis()
    ax.invert_yaxis()
    ax.yaxis.tick_right()
    plt.xlabel('1H (ppm)', **hfont, fontweight='bold')
    plt.ylabel('13C (ppm)', **hfont, fontweight='bold')
    ax.xaxis.set_tick_params(labelsize=10)
    ax.yaxis.set_tick_params(labelsize=10)
    
    

    fig3d = plt.figure()
    ax3d = plt.axes(projection='3d')
    ax3d.contour3D(X_fit, Y_fit, peak_data_fit, 50, cmap='summer')
    ax3d.plot_wireframe(X, Y, peak_data_red.values, color='red')
    
    plt.xlabel('1H (ppm)', **hfont, fontweight='bold')
    plt.ylabel('13C (ppm)', **hfont, fontweight='bold')
    
    # Save plots do disk.
    plt.title("HSQC: {} x: {}  y: {}  int: {} R2: {:2f}".format((h+1), x0, y0, intensity, fit_r2))
    fig_name = "{}_{}_{}.png".format(h, x0, y0)
    plt.savefig(fig_name, dpi=300)
    #plt.close(fig)

    

### Saving data

In [ ]:
# Save fitted peak parameters to disk.
param_names = ['h', 'x0', 'y0', 'Intensity', 'A', 'mu_x', 'mu_y', 'sigma_x', 'sigma_y', 'volume', 'ln(V)', 'r2','A', 'mu_x', 'mu_y', 'sigma_x', 'sigma_y', 'A', 'mu_x', 'mu_y', 'sigma_x', 'sigma_y','V0', 'BO4']
print(results)

### Processing extracted data

In [ ]:
# Analyse volume results
ln_V_0 = []
for n in range(1,x+1,1): #Adapt this number based on the number x of peaks you fitted: you have 3 spectra and x peaks in each spectra
    y = []
    for m in range (0, 3*x, 1): #Adapt this number based on the number x of peaks you fitted: you have 3 spectra and x peaks in each spectra
      if results[m][0] == n:
        y.append(results[m][10])
    ln_V_0.append(y)
print(ln_V_0)

V_0 = []
for p in range (0, x, 1): ): #Adapt this number based on the number x of peaks you fitted
  model = LinearRegression()
  y = np.array([ln_V_0[q][p] for q in range(0,3,1)])
  model.fit(HSQC.reshape(-1, 1), y.reshape(-1, 1))
  zero = model.intercept_
  V_0.append((zero))
print(V_0)


n_PS = m_PS / MW_PS #mole of polystyrene in the tube
n_BO4 = []  #mole of B-O-4 linkages in the tube
n_BO4_mass = [] #mole of B-O-4 linkages in the tube per g of lignin

for k in range (0, len(V_0) -1 ,1):
  n_BO4.append(n_PS*np.exp(V_0[k])/np.exp(V_0[k])) #k is the index corresponding to the PS peak
  n_BO4_mass.append((n_PS*np.exp(V_0[k])/np.exp(V_0[k]))/(m_lig*0.001)) #k is the index corresponding to the PS peak
print(n_BO4)
print('mass', n_BO4_mass)

for k in range (0, x-1, 1):
    results[k].append(str(V_0[k]))
    results[k].append(str(n_BO4_mass[k]))
fitted_params_df = pd.DataFrame(results, columns=param_names)
fitted_params_df.to_csv("fitted_parameters.csv", index=False)